<div class="text-warning">
    <div class="text-center">
        <div class="h1">practise 7 - google finance prediction</div>
        <a href='https://colab.research.google.com/drive/1wad2bpXUxxAq6CPFh6eTunC2HTNY53GV?usp=sharing'>google colab link</a>
        
        <div class="h5">maktabkhooneh</div>
    </div>
    <div class="h4">Name: Amir Mohammad Arghavany</div>
    <div class="h4">Email: amir.arghavoon@gmail.com</div>
</div>

# import packages

In [1]:
# base packages
import pandas as pd
import numpy as np
# visualization
import plotly.graph_objects as go
import plotly.express as px
# collect data
import yfinance as yf
# ML
from prophet import Prophet

# add stock name

In [2]:
stock_name=['google', 'GOOG']

# collect data

In [ ]:
df = yf.Ticker(stock_name[1]).history(period='max')

# plot candlestick

In [3]:
fig = go.Figure(
    data=[
        go.Candlestick(
            x=df.index,
            open=df["Open"],
            high=df["High"],
            low=df["Low"],
            close=df["Close"],
        )
    ]
)

fig.update_layout(
    margin=dict(l=20, r=20, t=50, b=20),
    height=600,
    title=stock_name[0],
)

fig.show()

In [4]:
df = df['Open']

In [5]:
df = df.reset_index()

# train & test

In [6]:
split_date = '2023-06-01'
train = df[df.Date <= split_date]
test = df[df.Date >= split_date]
print('test size:', np.round(len(test)/len(df)*100, 2), '%')
print('train size:', np.round(len(train)/len(df)*100, 2), '%')

test size: 4.14 %
train size: 95.88 %


In [7]:
train = train.rename(columns={
    'Date': 'ds',
    'Open': 'y'
})
test = test.rename(columns={
    'Date': 'ds',
    'Open': 'y'
})

In [8]:
train['ds'] = train['ds'].dt.tz_localize(None)
test['ds'] = test['ds'].dt.tz_localize(None)

# ML

In [9]:
model = Prophet()
model.fit(train)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqejp_s_j/v2c3at3y.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqejp_s_j/otlqz9e9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=17751', 'data', 'file=/tmp/tmpqejp_s_j/v2c3at3y.json', 'init=/tmp/tmpqejp_s_j/otlqz9e9.json', 'output', 'file=/tmp/tmpqejp_s_j/prophet_modelh95v1nqg/prophet_model-20240323123531.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:35:31 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:35:32 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


# predict

In [10]:
y_pred = model.predict(test)

# plot train & test & predict & lower predict & upper predict

In [11]:
fig1 = px.line(train, x="ds", y="y", title='train')
fig1.update_traces(line_color='black')

fig2 = px.line(test, x="ds", y="y", title='test')
fig2.update_traces(line_color='red')

fig3 = px.line(y_pred, x="ds", y="yhat", title='predicted')
fig3.update_traces(line_color='purple')
fig4 = px.line(y_pred, x="ds", y="yhat_lower", title='lower predicted')
fig4.update_traces(line_color='blue')
fig5 = px.line(y_pred, x="ds", y="yhat_upper", title='upper predicted')
fig5.update_traces(line_color='blue')

fig4 = go.Figure(data=fig1.data + fig2.data + fig3.data + fig4.data + fig5.data)
fig4.update_xaxes(title_text="Date")
fig4.update_yaxes(title_text="Open price")
fig4.update_layout(margin=dict(l=20, r=20, t=50, b=20), height=600, title=stock_name[0])

fig4.show()